In [23]:
import json
from openai import AzureOpenAI
import time
import os
from datetime import datetime

In [32]:
client  = AzureOpenAI(
    api_key = "fe96427f0ca049838844dea132dff8bc", #os.getenv("OPENAI_API_KEY"),
    azure_endpoint = "https://swedencentral.api.cognitive.microsoft.com/", #os.getenv("OPENAI_API_BASE"),
    api_version = "2024-02-15-preview" #os.getenv("OPENAI_API_VERSION")
)

In [33]:
tools_list = [
    {
        "type": "code_interpreter"
    }
]

instructions = (
    "You are a sales analysis assistant for contoso.",
    "Please be polite, professional, helpful, and friendly.",
    "You get all the sales data from this app using the functions provided.",
    "If question is not related to the sales or you cannot answer the question, ",
    "say, 'contact IT for more assistance.' ",
    "If the user asks for help or says 'help', provide a list of sample questions that you can answer.",
    f"The current date and time is: {datetime.now().strftime('%x %X')}.",
)

assistant = client.beta.assistants.create(
    name="Portfolio Management Assistant",
    model="gpt-4o",
    instructions=str(instructions),
    tools=tools_list,
)

In [34]:
print(assistant.id)

asst_0otHx9MEqlWgwXl5KWKdnSZQ


In [35]:
thread = client.beta.threads.create()

In [36]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="hi" # Replace this with your prompt
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [38]:
# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print("ja")
  print(messages)
elif run.status == 'requires_action':
  # the assistant requires calling some functions
  # and submit the tool outputs back to the run
  pass
else:
  print(run.status)

ja
SyncCursorPage[Message](data=[Message(id='msg_fAUg4IWAgSZWjapf5LyC3JYT', assistant_id='asst_0otHx9MEqlWgwXl5KWKdnSZQ', attachments=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Hello! How can I assist you today with your sales data?'), type='text')], created_at=1723042020, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_uKqfvxvjoCHoib16foBTZiW2', status=None, thread_id='thread_d6uRrMTUPAIp9CFLkKeYBChr', file_ids=[]), Message(id='msg_FQHoSfiyHXWWIlxUrEnPxnqV', assistant_id=None, attachments=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='hi'), type='text')], created_at=1723042006, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_d6uRrMTUPAIp9CFLkKeYBChr', file_ids=[])], object='list', first_id='msg_fAUg4IWAgSZWjapf5LyC3JYT', last_id='msg_FQHoSfiyHXWWIlxUrEnP